In [1]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta
from ipywidgets import interact, IntSlider
from IPython.display import display
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
# import seaborn as sns
# from matplotlib import pyplot as plt
# %matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

from pycaret.classification import * 

# pd.options.display.max_columns = 5000
# pd.options.display.max_rows = 5000

injured_pred = 'injured_in_52_week'

simple_features = ['Height', 'Weight', 'age','cum_injury_total', 'weeks_since_last_injury', 'Min_cum','Serie A_cum',
 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', 'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum',
 'Copa América_cum', 'Away_cum', 'Home_cum', 'Neutral_cum']

extended_features = ['Height', 'Weight', 'defender', 'attacker', 'midfielder', 'goalkeeper', 'right_foot', 'age', 'cum_injury_total', 'weeks_since_last_injury', 'Min_cum', 'Gls_cum', 'Ast_cum', 'PK_cum', 'PKatt_cum',
 'Sh_cum', 'SoT_cum', 'CrdY_cum', 'CrdR_cum', 'Touches_cum', 'Press_cum', 'Tkl_cum', 'Int_cum', 'Blocks_cum', 'xG_cum', 'npxG_cum', 'xA_cum', 'SCA_cum', 'GCA_cum', 'Cmp_cum',
 'Att_cum', 'Prog_cum', 'Carries_cum', 'Prog.1_cum', 'Succ_cum', 'Att.1_cum', 'Fls_cum', 'Fld_cum', 'Off_cum', 'Crs_cum', 'TklW_cum', 'OG_cum', 'PKwon_cum','PKcon_cum', 'Serie A_cum',
 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', 'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum',
 'Copa América_cum', 'Away_cum', 'Home_cum', 'Neutral_cum']

In [2]:
# Load Dataset

# Renzo's Path
# dataset = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/complete_final_df_5.csv')
#
# Sravan's Path
# dataset = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/dataset_for_model_final.csv')

# Sravan's Parquet Path
dataset = pd.read_parquet('dataframes_blog/dataset_for_model_final.parquet') 

In [ ]:
# dataset[dataset['FBRefID']=='dea698d9'].to_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/ronaldo.csv', index=False)
dataset = dataset[dataset['drop'] == 0]
dataset = dataset[dataset['goalkeeper'] == 0]

In [ ]:
# Get Train Test Split
df_train = dataset[dataset['cum_week'] <= dataset["train_split"]].dropna()
df_test = dataset[dataset['cum_week'] > dataset["train_split"]].dropna()

In [ ]:
X_train = df_train[extended_features]
y_train = df_train[injured_pred]

X_test = df_test[extended_features]
y_test = df_test[injured_pred]

In [ ]:
exp_clf = setup(dataset[extended_features + [injured_pred]], target=injured_pred, fix_imbalance=True, feature_selection=True, fold=2)

In [7]:
set_config('X_train', X_train)
set_config('X_test', X_test)
set_config('y_train', y_train)
set_config('y_test', y_test)

In [8]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.9282,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,3.7100
rf,Random Forest Classifier,0.9127,0.5663,0.0243,0.0923,0.0385,0.0088,0.0110,23.1850
lightgbm,Light Gradient Boosting Machine,0.9039,0.5986,0.0506,0.1161,0.0704,0.0279,0.0306,17.1050
et,Extra Trees Classifier,0.8819,0.5646,0.0740,0.0934,0.0826,0.0204,0.0205,23.9500
ridge,Ridge Classifier,0.8370,0.0000,0.2233,0.1301,0.1645,0.0811,0.0847,4.3400
lda,Linear Discriminant Analysis,0.8370,0.6136,0.2233,0.1301,0.1644,0.0811,0.0847,8.8450
gbc,Gradient Boosting Classifier,0.8345,0.6419,0.2455,0.1367,0.1756,0.0919,0.0967,302.7750
dt,Decision Tree Classifier,0.7823,0.5104,0.1951,0.0806,0.1140,0.0138,0.0156,20.3300
ada,Ada Boost Classifier,0.7520,0.6318,0.3903,0.1207,0.1843,0.0839,0.1041,79.3400
knn,K Neighbors Classifier,0.7232,0.5384,0.2958,0.0859,0.1331,0.0245,0.0312,509.8500


In [ ]:
# df = pull()
# df.to_csv('results_1_year.csv', index=False)
df = pull().sort_values(by=['F1'], ascending=False)

In [10]:
df

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.7520,0.6318,0.3903,0.1207,0.1843,0.0839,0.1041,79.340
gbc,Gradient Boosting Classifier,0.8345,0.6419,0.2455,0.1367,0.1756,0.0919,0.0967,302.775
lr,Logistic Regression,0.6738,0.5990,0.4502,0.1013,0.1654,0.0546,0.0783,122.505
ridge,Ridge Classifier,0.8370,0.0000,0.2233,0.1301,0.1645,0.0811,0.0847,4.340
lda,Linear Discriminant Analysis,0.8370,0.6136,0.2233,0.1301,0.1644,0.0811,0.0847,8.845
nb,Naive Bayes,0.4668,0.5831,0.6314,0.0874,0.1506,0.0293,0.0477,4.845
knn,K Neighbors Classifier,0.7232,0.5384,0.2958,0.0859,0.1331,0.0245,0.0312,509.850
svm,SVM - Linear Kernel,0.5622,0.0000,0.4735,0.1168,0.1173,0.0311,0.0457,96.580
dt,Decision Tree Classifier,0.7823,0.5104,0.1951,0.0806,0.1140,0.0138,0.0156,20.330
qda,Quadratic Discriminant Analysis,0.6751,0.5681,0.3238,0.0830,0.0917,0.0100,0.0172,7.375


In [12]:
# Select best model
model = create_model(df.index[0], fold=5)
save_model(model, 'model_1_year')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7359,0.6265,0.4045,0.1160,0.1803,0.0773,0.0990
1,0.7256,0.6360,0.4322,0.1173,0.1845,0.0806,0.1056
2,0.7545,0.6208,0.3624,0.1153,0.1749,0.0740,0.0909
3,0.7362,0.6308,0.4039,0.1160,0.1802,0.0773,0.0988
4,0.7345,0.6389,0.4006,0.1145,0.1781,0.0748,0.0958
Mean,0.7373,0.6306,0.4007,0.1158,0.1796,0.0768,0.0980
SD,0.0094,0.0065,0.0223,0.0009,0.0031,0.0023,0.0048


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='injured_in_26_week',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 n...
                  Advanced_Feature_Selection_Classic(ml_usecase='classification',
                                                     n_jobs=-1,
                                                     random_state=8301,
                 

In [14]:
tuned_model = tune_model(model, optimize = 'F1')
save_model(tuned_model, 'model_1_year_tuned')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7363,0.6396,0.4127,0.1180,0.1835,0.0809,0.1036
1,0.7292,0.6422,0.4297,0.1183,0.1856,0.0822,0.1070
Mean,0.7327,0.6409,0.4212,0.1182,0.1845,0.0815,0.1053
SD,0.0036,0.0013,0.0085,0.0002,0.0010,0.0007,0.0017


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='injured_in_26_week',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 n...
                  Advanced_Feature_Selection_Classic(ml_usecase='classification',
                                                     n_jobs=-1,
                                                     random_state=8301,
                 

In [3]:
# Loading the saved model
model_1_year_tuned = load_model('model_1_year_tuned')

Transformation Pipeline and Model Successfully Loaded


In [4]:
model_1_year_tuned

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(ml_usecase='classification',
                                      target='injured_in_52_week')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         'fr...
                ('fix_perfect', Remove_100(target='injured_in_52_week')),
                ('clean_names', Clean_Colum_Names()),
                ('feature_select',
                 Advanced_Feature_Selection_Classic(n_jobs=-1,
                                                    random_state=2889,
               